In [1]:
import pandas as pd
import glob
from os.path import basename

In [2]:
CURSO = 'F55'
RUTA_GENERAL = './INPUT/GRUPOS_PREDEFINITIVAS/'
RUTA_ESPECIFICA = RUTA_GENERAL + CURSO
PREFIJO_CUESTIONARIOS = 'Cuestionario:'
SUFIJO_ACTIVIDADES = ' (Real)'
NOMENCLATURA_OMITIR = 5
TOLERANCIA_MOODLE = 0.051
ACTIVIDADES_CALIFICABLES = [
    'Academic Reading 2',    
    'Academic Reading 4',
    'Assessment Test - Cycle 1',
    'ESP Reading 2',
    'ESP Reading 4',
    'Mid-Term Exam'
]
BLOQUES_ACTIVIDADES = {
    'Total Activities 1 (Real)' : ['ESP Reading 2', 'Academic Reading 2'],
    'Total Mid Term Exam (Real)' : ['Mid-Term Exam'],
    'Total Activities 2 (Real)': ['Academic Reading 4', 'ESP Reading 4'],
    'Total Final Exam (Real)': ['Assessment Test - Cycle 1']
}

In [3]:
def extraerNombre(ruta):
    nBase = basename(archivo) 
    tokens = nBase.split('-')
    casiNombre = '-'.join(tokens[NOMENCLATURA_OMITIR:-1])
    nombre = casiNombre.split('.')[0]
    return nombre
def generarNombreColumnaLibro(nombre):
    return PREFIJO_CUESTIONARIOS + nombre + SUFIJO_ACTIVIDADES

def getDfActividadEstudiante(dfs, actividad, codigo):
    dfActividad = dfs[actividad]
    dfActividad = dfActividad[dfActividad['Número de ID'] == codigo]
    return dfActividad
def getNotaActividad(dfActividad, rayita = True):
    columnaCalificacion = dfActividad.columns[8]
    if len(dfActividad) > 0:
        return float(dfActividad[columnaCalificacion].iloc[0])
    return '-' if rayita else 0
def getNotaLibro(notaLibro, rayita = True):
    if notaLibro == '-':
        return notaLibro if rayita else 0.0
    return float(notaLibro)

In [4]:
archivos = glob.glob(RUTA_ESPECIFICA + '/*.csv')
dfs = {}
bandera = False
for archivo in archivos:
    nombre = extraerNombre(archivo)
    if nombre in ACTIVIDADES_CALIFICABLES:
        df = pd.read_csv(archivo)
        df = df.drop(len(df) - 1) # Quita el promedio general
        columnaCalificacion = df.columns[8]
        df = df.sort_values(columnaCalificacion)
        test = df[df[columnaCalificacion] == '-']
        if len(test) > 0:
            bandera = True
            print ('\nSE ENCONTRARON ESTUDIANTES SIN ENVIAR: ')
            print(test[['Nombre', 'Apellido(s)', 'Número de ID','Estado']])
        df = df.drop_duplicates('Nombre de usuario', keep = 'last') # Quita todos los intentos y deja el más alto
        dfs[nombre] = df
    else:
        dfs['LIBRO'] = pd.read_csv(archivo)

In [5]:
for indexLibro, rowLibro in dfs['LIBRO'].iterrows():
    if bandera:
        print ('Hay notas sin enviar')
        break
    codEst = rowLibro['Número de ID']
    for actividad in ACTIVIDADES_CALIFICABLES:
        calificacionLibro = getNotaLibro(rowLibro[generarNombreColumnaLibro(actividad)])
        dfActividad = getDfActividadEstudiante(dfs, actividad, codEst)
        calificacionAct = getNotaActividad(dfActividad)        
        if calificacionLibro != calificacionAct and not(calificacionLibro <= calificacionAct <= calificacionLibro + TOLERANCIA_MOODLE):
            print ('=============')
            print ('\nINCONSISTENCIA DETECTADA', codEst, actividad)
            print ('Calificación libro: ', calificacionLibro,'Calificacion actividad: ', calificacionAct)
            #print ((calificacionLibro <= calificacionAct <= calificacionLibro + TOLERANCIA_MOODLE))
            #print (calificacionLibro, calificacionAct, calificacionLibro + TOLERANCIA_MOODLE)
    
    # Verificación de notas en bloques de actividades
    for carpeta, items in BLOQUES_ACTIVIDADES.items():
        notasCarpeta = 0
        for actividad in items:
            dfActividad = getDfActividadEstudiante(dfs, actividad, codEst)
            notasCarpeta += getNotaActividad(dfActividad, rayita = False)
        notaCarpetaLibro = getNotaLibro(rowLibro[carpeta], False) 
        notaCarpeta = notasCarpeta/len(items)
        diff = abs(notaCarpeta - notaCarpetaLibro)
        if diff > TOLERANCIA_MOODLE:
            print ('++++++++++++')
            print ('\nINCONSISTENCIA EN CARPETA DETECTADA', codEst, carpeta)
            print ('Calificación libro: ', notaCarpetaLibro,'Calificacion carpeta: ', notaCarpeta)
            